In [24]:
import os
import gc
import math
import random
import datetime
from time import time

from tqdm import tqdm
import pandas as pd
import numpy as np
from numba import njit
from functools import partial
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier

from scipy.optimize import fmin, minimize, fsolve
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.loss import _WeightedLoss
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning import Callback
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

import lightgbm as lgb
import xgboost as xgb

# import optuna
# from optuna.integration import PyTorchLightningPruningCallback

import tensorboard

%matplotlib inline
%load_ext tensorboard
pd.options.display.max_columns = None

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
SEED                = 42
TRAIN_COLUMNS       = ["sig_id", "cp_type", "cp_time", "cp_dose", "g-0", "g-1", "g-2", "g-3", "g-4", "g-5", "g-6", "g-7", "g-8", "g-9", "g-10", "g-11", "g-12", "g-13", "g-14", "g-15", "g-16", "g-17", "g-18", "g-19", "g-20", "g-21", "g-22", "g-23", "g-24", "g-25", "g-26", "g-27", "g-28", "g-29", "g-30", "g-31", "g-32", "g-33", "g-34", "g-35", "g-36", "g-37", "g-38", "g-39", "g-40", "g-41", "g-42", "g-43", "g-44", "g-45", "g-46", "g-47", "g-48", "g-49", "g-50", "g-51", "g-52", "g-53", "g-54", "g-55", "g-56", "g-57", "g-58", "g-59", "g-60", "g-61", "g-62", "g-63", "g-64", "g-65", "g-66", "g-67", "g-68", "g-69", "g-70", "g-71", "g-72", "g-73", "g-74", "g-75", "g-76", "g-77", "g-78", "g-79", "g-80", "g-81", "g-82", "g-83", "g-84", "g-85", "g-86", "g-87", "g-88", "g-89", "g-90", "g-91", "g-92", "g-93", "g-94", "g-95", "g-96", "g-97", "g-98", "g-99", "g-100", "g-101", "g-102", "g-103", "g-104", "g-105", "g-106", "g-107", "g-108", "g-109", "g-110", "g-111", "g-112", "g-113", "g-114", "g-115", "g-116", "g-117", "g-118", "g-119", "g-120", "g-121", "g-122", "g-123", "g-124", "g-125", "g-126", "g-127", "g-128", "g-129", "g-130", "g-131", "g-132", "g-133", "g-134", "g-135", "g-136", "g-137", "g-138", "g-139", "g-140", "g-141", "g-142", "g-143", "g-144", "g-145", "g-146", "g-147", "g-148", "g-149", "g-150", "g-151", "g-152", "g-153", "g-154", "g-155", "g-156", "g-157", "g-158", "g-159", "g-160", "g-161", "g-162", "g-163", "g-164", "g-165", "g-166", "g-167", "g-168", "g-169", "g-170", "g-171", "g-172", "g-173", "g-174", "g-175", "g-176", "g-177", "g-178", "g-179", "g-180", "g-181", "g-182", "g-183", "g-184", "g-185", "g-186", "g-187", "g-188", "g-189", "g-190", "g-191", "g-192", "g-193", "g-194", "g-195", "g-196", "g-197", "g-198", "g-199", "g-200", "g-201", "g-202", "g-203", "g-204", "g-205", "g-206", "g-207", "g-208", "g-209", "g-210", "g-211", "g-212", "g-213", "g-214", "g-215", "g-216", "g-217", "g-218", "g-219", "g-220", "g-221", "g-222", "g-223", "g-224", "g-225", "g-226", "g-227", "g-228", "g-229", "g-230", "g-231", "g-232", "g-233", "g-234", "g-235", "g-236", "g-237", "g-238", "g-239", "g-240", "g-241", "g-242", "g-243", "g-244", "g-245", "g-246", "g-247", "g-248", "g-249", "g-250", "g-251", "g-252", "g-253", "g-254", "g-255", "g-256", "g-257", "g-258", "g-259", "g-260", "g-261", "g-262", "g-263", "g-264", "g-265", "g-266", "g-267", "g-268", "g-269", "g-270", "g-271", "g-272", "g-273", "g-274", "g-275", "g-276", "g-277", "g-278", "g-279", "g-280", "g-281", "g-282", "g-283", "g-284", "g-285", "g-286", "g-287", "g-288", "g-289", "g-290", "g-291", "g-292", "g-293", "g-294", "g-295", "g-296", "g-297", "g-298", "g-299", "g-300", "g-301", "g-302", "g-303", "g-304", "g-305", "g-306", "g-307", "g-308", "g-309", "g-310", "g-311", "g-312", "g-313", "g-314", "g-315", "g-316", "g-317", "g-318", "g-319", "g-320", "g-321", "g-322", "g-323", "g-324", "g-325", "g-326", "g-327", "g-328", "g-329", "g-330", "g-331", "g-332", "g-333", "g-334", "g-335", "g-336", "g-337", "g-338", "g-339", "g-340", "g-341", "g-342", "g-343", "g-344", "g-345", "g-346", "g-347", "g-348", "g-349", "g-350", "g-351", "g-352", "g-353", "g-354", "g-355", "g-356", "g-357", "g-358", "g-359", "g-360", "g-361", "g-362", "g-363", "g-364", "g-365", "g-366", "g-367", "g-368", "g-369", "g-370", "g-371", "g-372", "g-373", "g-374", "g-375", "g-376", "g-377", "g-378", "g-379", "g-380", "g-381", "g-382", "g-383", "g-384", "g-385", "g-386", "g-387", "g-388", "g-389", "g-390", "g-391", "g-392", "g-393", "g-394", "g-395", "g-396", "g-397", "g-398", "g-399", "g-400", "g-401", "g-402", "g-403", "g-404", "g-405", "g-406", "g-407", "g-408", "g-409", "g-410", "g-411", "g-412", "g-413", "g-414", "g-415", "g-416", "g-417", "g-418", "g-419", "g-420", "g-421", "g-422", "g-423", "g-424", "g-425", "g-426", "g-427", "g-428", "g-429", "g-430", "g-431", "g-432", "g-433", "g-434", "g-435", "g-436", "g-437", "g-438", "g-439", "g-440", "g-441", "g-442", "g-443", "g-444", "g-445", "g-446", "g-447", "g-448", "g-449", "g-450", "g-451", "g-452", "g-453", "g-454", "g-455", "g-456", "g-457", "g-458", "g-459", "g-460", "g-461", "g-462", "g-463", "g-464", "g-465", "g-466", "g-467", "g-468", "g-469", "g-470", "g-471", "g-472", "g-473", "g-474", "g-475", "g-476", "g-477", "g-478", "g-479", "g-480", "g-481", "g-482", "g-483", "g-484", "g-485", "g-486", "g-487", "g-488", "g-489", "g-490", "g-491", "g-492", "g-493", "g-494", "g-495", "g-496", "g-497", "g-498", "g-499", "g-500", "g-501", "g-502", "g-503", "g-504", "g-505", "g-506", "g-507", "g-508", "g-509", "g-510", "g-511", "g-512", "g-513", "g-514", "g-515", "g-516", "g-517", "g-518", "g-519", "g-520", "g-521", "g-522", "g-523", "g-524", "g-525", "g-526", "g-527", "g-528", "g-529", "g-530", "g-531", "g-532", "g-533", "g-534", "g-535", "g-536", "g-537", "g-538", "g-539", "g-540", "g-541", "g-542", "g-543", "g-544", "g-545", "g-546", "g-547", "g-548", "g-549", "g-550", "g-551", "g-552", "g-553", "g-554", "g-555", "g-556", "g-557", "g-558", "g-559", "g-560", "g-561", "g-562", "g-563", "g-564", "g-565", "g-566", "g-567", "g-568", "g-569", "g-570", "g-571", "g-572", "g-573", "g-574", "g-575", "g-576", "g-577", "g-578", "g-579", "g-580", "g-581", "g-582", "g-583", "g-584", "g-585", "g-586", "g-587", "g-588", "g-589", "g-590", "g-591", "g-592", "g-593", "g-594", "g-595", "g-596", "g-597", "g-598", "g-599", "g-600", "g-601", "g-602", "g-603", "g-604", "g-605", "g-606", "g-607", "g-608", "g-609", "g-610", "g-611", "g-612", "g-613", "g-614", "g-615", "g-616", "g-617", "g-618", "g-619", "g-620", "g-621", "g-622", "g-623", "g-624", "g-625", "g-626", "g-627", "g-628", "g-629", "g-630", "g-631", "g-632", "g-633", "g-634", "g-635", "g-636", "g-637", "g-638", "g-639", "g-640", "g-641", "g-642", "g-643", "g-644", "g-645", "g-646", "g-647", "g-648", "g-649", "g-650", "g-651", "g-652", "g-653", "g-654", "g-655", "g-656", "g-657", "g-658", "g-659", "g-660", "g-661", "g-662", "g-663", "g-664", "g-665", "g-666", "g-667", "g-668", "g-669", "g-670", "g-671", "g-672", "g-673", "g-674", "g-675", "g-676", "g-677", "g-678", "g-679", "g-680", "g-681", "g-682", "g-683", "g-684", "g-685", "g-686", "g-687", "g-688", "g-689", "g-690", "g-691", "g-692", "g-693", "g-694", "g-695", "g-696", "g-697", "g-698", "g-699", "g-700", "g-701", "g-702", "g-703", "g-704", "g-705", "g-706", "g-707", "g-708", "g-709", "g-710", "g-711", "g-712", "g-713", "g-714", "g-715", "g-716", "g-717", "g-718", "g-719", "g-720", "g-721", "g-722", "g-723", "g-724", "g-725", "g-726", "g-727", "g-728", "g-729", "g-730", "g-731", "g-732", "g-733", "g-734", "g-735", "g-736", "g-737", "g-738", "g-739", "g-740", "g-741", "g-742", "g-743", "g-744", "g-745", "g-746", "g-747", "g-748", "g-749", "g-750", "g-751", "g-752", "g-753", "g-754", "g-755", "g-756", "g-757", "g-758", "g-759", "g-760", "g-761", "g-762", "g-763", "g-764", "g-765", "g-766", "g-767", "g-768", "g-769", "g-770", "g-771", "c-0", "c-1", "c-2", "c-3", "c-4", "c-5", "c-6", "c-7", "c-8", "c-9", "c-10", "c-11", "c-12", "c-13", "c-14", "c-15", "c-16", "c-17", "c-18", "c-19", "c-20", "c-21", "c-22", "c-23", "c-24", "c-25", "c-26", "c-27", "c-28", "c-29", "c-30", "c-31", "c-32", "c-33", "c-34", "c-35", "c-36", "c-37", "c-38", "c-39", "c-40", "c-41", "c-42", "c-43", "c-44", "c-45", "c-46", "c-47", "c-48", "c-49", "c-50", "c-51", "c-52", "c-53", "c-54", "c-55", "c-56", "c-57", "c-58", "c-59", "c-60", "c-61", "c-62", "c-63", "c-64", "c-65", "c-66", "c-67", "c-68", "c-69", "c-70", "c-71", "c-72", "c-73", "c-74", "c-75", "c-76", "c-77", "c-78", "c-79", "c-80", "c-81", "c-82", "c-83", "c-84", "c-85", "c-86", "c-87", "c-88", "c-89", "c-90", "c-91", "c-92", "c-93", "c-94", "c-95", "c-96", "c-97", "c-98", "c-99"]
TARGET_COLUMNS      = ['5-alpha_reductase_inhibitor', '11-beta-hsd1_inhibitor', 'acat_inhibitor', 'acetylcholine_receptor_agonist', 'acetylcholine_receptor_antagonist', 'acetylcholinesterase_inhibitor', 'adenosine_receptor_agonist', 'adenosine_receptor_antagonist', 'adenylyl_cyclase_activator', 'adrenergic_receptor_agonist', 'adrenergic_receptor_antagonist', 'akt_inhibitor', 'aldehyde_dehydrogenase_inhibitor', 'alk_inhibitor', 'ampk_activator', 'analgesic', 'androgen_receptor_agonist', 'androgen_receptor_antagonist', 'anesthetic_-_local', 'angiogenesis_inhibitor', 'angiotensin_receptor_antagonist', 'anti-inflammatory', 'antiarrhythmic', 'antibiotic', 'anticonvulsant', 'antifungal', 'antihistamine', 'antimalarial', 'antioxidant', 'antiprotozoal', 'antiviral', 'apoptosis_stimulant', 'aromatase_inhibitor', 'atm_kinase_inhibitor', 'atp-sensitive_potassium_channel_antagonist', 'atp_synthase_inhibitor', 'atpase_inhibitor', 'atr_kinase_inhibitor', 'aurora_kinase_inhibitor', 'autotaxin_inhibitor', 'bacterial_30s_ribosomal_subunit_inhibitor', 'bacterial_50s_ribosomal_subunit_inhibitor', 'bacterial_antifolate', 'bacterial_cell_wall_synthesis_inhibitor', 'bacterial_dna_gyrase_inhibitor', 'bacterial_dna_inhibitor', 'bacterial_membrane_integrity_inhibitor', 'bcl_inhibitor', 'bcr-abl_inhibitor', 'benzodiazepine_receptor_agonist', 'beta_amyloid_inhibitor', 'bromodomain_inhibitor', 'btk_inhibitor', 'calcineurin_inhibitor', 'calcium_channel_blocker', 'cannabinoid_receptor_agonist', 'cannabinoid_receptor_antagonist', 'carbonic_anhydrase_inhibitor', 'casein_kinase_inhibitor', 'caspase_activator', 'catechol_o_methyltransferase_inhibitor', 'cc_chemokine_receptor_antagonist', 'cck_receptor_antagonist', 'cdk_inhibitor', 'chelating_agent', 'chk_inhibitor', 'chloride_channel_blocker', 'cholesterol_inhibitor', 'cholinergic_receptor_antagonist', 'coagulation_factor_inhibitor', 'corticosteroid_agonist', 'cyclooxygenase_inhibitor', 'cytochrome_p450_inhibitor', 'dihydrofolate_reductase_inhibitor', 'dipeptidyl_peptidase_inhibitor', 'diuretic', 'dna_alkylating_agent', 'dna_inhibitor', 'dopamine_receptor_agonist', 'dopamine_receptor_antagonist', 'egfr_inhibitor', 'elastase_inhibitor', 'erbb2_inhibitor', 'estrogen_receptor_agonist', 'estrogen_receptor_antagonist', 'faah_inhibitor', 'farnesyltransferase_inhibitor', 'fatty_acid_receptor_agonist', 'fgfr_inhibitor', 'flt3_inhibitor', 'focal_adhesion_kinase_inhibitor', 'free_radical_scavenger', 'fungal_squalene_epoxidase_inhibitor', 'gaba_receptor_agonist', 'gaba_receptor_antagonist', 'gamma_secretase_inhibitor', 'glucocorticoid_receptor_agonist', 'glutamate_inhibitor', 'glutamate_receptor_agonist', 'glutamate_receptor_antagonist', 'gonadotropin_receptor_agonist', 'gsk_inhibitor', 'hcv_inhibitor', 'hdac_inhibitor', 'histamine_receptor_agonist', 'histamine_receptor_antagonist', 'histone_lysine_demethylase_inhibitor', 'histone_lysine_methyltransferase_inhibitor', 'hiv_inhibitor', 'hmgcr_inhibitor', 'hsp_inhibitor', 'igf-1_inhibitor', 'ikk_inhibitor', 'imidazoline_receptor_agonist', 'immunosuppressant', 'insulin_secretagogue', 'insulin_sensitizer', 'integrin_inhibitor', 'jak_inhibitor', 'kit_inhibitor', 'laxative', 'leukotriene_inhibitor', 'leukotriene_receptor_antagonist', 'lipase_inhibitor', 'lipoxygenase_inhibitor', 'lxr_agonist', 'mdm_inhibitor', 'mek_inhibitor', 'membrane_integrity_inhibitor', 'mineralocorticoid_receptor_antagonist', 'monoacylglycerol_lipase_inhibitor', 'monoamine_oxidase_inhibitor', 'monopolar_spindle_1_kinase_inhibitor', 'mtor_inhibitor', 'mucolytic_agent', 'neuropeptide_receptor_antagonist', 'nfkb_inhibitor', 'nicotinic_receptor_agonist', 'nitric_oxide_donor', 'nitric_oxide_production_inhibitor', 'nitric_oxide_synthase_inhibitor', 'norepinephrine_reuptake_inhibitor', 'nrf2_activator', 'opioid_receptor_agonist', 'opioid_receptor_antagonist', 'orexin_receptor_antagonist', 'p38_mapk_inhibitor', 'p-glycoprotein_inhibitor', 'parp_inhibitor', 'pdgfr_inhibitor', 'pdk_inhibitor', 'phosphodiesterase_inhibitor', 'phospholipase_inhibitor', 'pi3k_inhibitor', 'pkc_inhibitor', 'potassium_channel_activator', 'potassium_channel_antagonist', 'ppar_receptor_agonist', 'ppar_receptor_antagonist', 'progesterone_receptor_agonist', 'progesterone_receptor_antagonist', 'prostaglandin_inhibitor', 'prostanoid_receptor_antagonist', 'proteasome_inhibitor', 'protein_kinase_inhibitor', 'protein_phosphatase_inhibitor', 'protein_synthesis_inhibitor', 'protein_tyrosine_kinase_inhibitor', 'radiopaque_medium', 'raf_inhibitor', 'ras_gtpase_inhibitor', 'retinoid_receptor_agonist', 'retinoid_receptor_antagonist', 'rho_associated_kinase_inhibitor', 'ribonucleoside_reductase_inhibitor', 'rna_polymerase_inhibitor', 'serotonin_receptor_agonist', 'serotonin_receptor_antagonist', 'serotonin_reuptake_inhibitor', 'sigma_receptor_agonist', 'sigma_receptor_antagonist', 'smoothened_receptor_antagonist', 'sodium_channel_inhibitor', 'sphingosine_receptor_agonist', 'src_inhibitor', 'steroid', 'syk_inhibitor', 'tachykinin_antagonist', 'tgf-beta_receptor_inhibitor', 'thrombin_inhibitor', 'thymidylate_synthase_inhibitor', 'tlr_agonist', 'tlr_antagonist', 'tnf_inhibitor', 'topoisomerase_inhibitor', 'transient_receptor_potential_channel_antagonist', 'tropomyosin_receptor_kinase_inhibitor', 'trpv_agonist', 'trpv_antagonist', 'tubulin_inhibitor', 'tyrosine_kinase_inhibitor', 'ubiquitin_specific_protease_inhibitor', 'vegfr_inhibitor', 'vitamin_b', 'vitamin_d_receptor_agonist', 'wnt_inhibitor']
GENES               = [col for col in TRAIN_COLUMNS if col.startswith('g-')]
CELLS               = [col for col in TRAIN_COLUMNS if col.startswith('c-')]

GENE_PCA_COMP       = int(0.6 * 772)
CELL_PCA_COMP       = int(0.6 * 100)
VARIANCE_THRESHOLD  = 0.9
N_CLUSTERS_G        = 22
N_CLUSTERS_C        = 4
FOLDS               = 7

INPUT_SIZE          = None
OUTPUT_SIZE         = None

SAVE_FOLDS          = True
USE_SAVED_FOLDS     = True
VERBOSE             = True

PATH                = "../data/"
TRAIN_F             = os.path.join(PATH, "train_features.csv")
TRAIN_T             = os.path.join(PATH, "train_targets_scored.csv")
TRAIN_T_NS          = os.path.join(PATH, "train_targets_nonscored.csv")
TEST_F              = os.path.join(PATH, "test_features.csv")
SAMPLE_SUBMISSION   = os.path.join(PATH, "sample_submission.csv")
PRE_FEATURES_CSV    = os.path.join(PATH, "preprocessed_train_features.csv")
PRE_TARGETS_CSV     = os.path.join(PATH, "preprocessed_train_targets.csv")
PRE_TEST_CSV        = os.path.join(PATH, "preprocessed_test_features.csv")
PRE_CP_FEATURES_CSV = os.path.join(PATH, "preprocessed_cp_train_features.csv")
PRE_CP_TARGETS_CSV  = os.path.join(PATH, "preprocessed_cp_train_targets.csv")

seed_everything(SEED)

42

In [3]:
class MoADatasetTrain(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return self.features.shape[0]
        
    def __getitem__(self, index):
        return (torch.tensor(self.features[index, :], dtype=torch.float), 
                    torch.tensor(self.targets[index, :], dtype=torch.float))
        
class MoADatasetTest(Dataset):
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return self.features.shape[0]
        
    def __getitem__(self, index):
        return torch.tensor(self.features[index, :], dtype=torch.float)

In [4]:
class MoADatasetModule(pl.LightningDataModule):
    def __init__(self, f, t, test_f, fold=0):
        super().__init__()
        self.f = f
        self.t = t
        self.test_f = test_f
        self.fold = fold
        self.batch_size = BATCH_SIZE

    def setup(self, stage=None):
        global INPUT_SIZE, OUTPUT_SIZE
        self.train_X = self.f[self.f.kfold != self.fold].drop(['kfold', 'sig_id'], axis=1).to_numpy().astype('float64')
        self.train_Y = self.t[self.f.kfold != self.fold].drop(['sig_id'], axis=1).to_numpy().astype('float64')
        
        INPUT_SIZE = self.train_X.shape[1]
        OUTPUT_SIZE = self.train_Y.shape[1]
        
        print(f"Dataset has {INPUT_SIZE} features.")
        print(f"Dataset has {OUTPUT_SIZE} targets.")
        
        self.valid_X = self.f[self.f.kfold == self.fold].drop(['kfold', 'sig_id'], axis=1).to_numpy().astype('float64')
        self.valid_Y = self.t[self.f.kfold == self.fold].drop(['sig_id'], axis=1).to_numpy().astype('float64')        
        self.test_X = self.test_f.to_numpy()
        
        if VERBOSE:
            print("Dataset Setup.")
            
        gc.collect()
        
    def train_dataloader(self):
        if VERBOSE:
            print("Train Dataloader Loaded.")
        return DataLoader(MoADatasetTrain(self.train_X, self.train_Y), self.batch_size, num_workers=0, shuffle=True)
    
    def val_dataloader(self):
        if VERBOSE:
            print("Validation Dataloader Loaded.")
        return DataLoader(MoADatasetTrain(self.valid_X, self.valid_Y), self.batch_size, num_workers=0, shuffle=False)

In [5]:
class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1 - y_true) * np.log(1 - logits + 1e-15) + y_true * np.log(logits + 1e-15)
        return np.mean(-aux)

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0, pos_weight=None):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        if pos_weight is not None:
            self.pos_weight = torch.tensor(pos_weight).cuda()
        else:
            self.pos_weight = pos_weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
                                           self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets, self.weight, pos_weight=self.pos_weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [6]:
# Models

class MoANet1(nn.Module):
    name = "MoANet1"
    def __init__(self, num_features, num_targets, trial=None):
        super().__init__()
        
        self.LAYER_OUTPUTS = [2048, 4096, 2048, 1024, 512]
        self.DROPOUT = 0.5
        self.NUM_LAYERS = 5
        
        if trial is not None:
            f_dropout = trial.suggest_float('f_dropout', 0.2, 0.5)
        else:
            f_dropout = self.DROPOUT
        
        layers = []

        # Intermediate layers
        in_size = num_features
        for i in range(self.NUM_LAYERS):
            out_size = self.LAYER_OUTPUTS[i]
#             out_size = trial.suggest_int('n_units_{}'.format(i), 256, 4096)
            layers.append(nn.utils.weight_norm(torch.nn.Linear(in_size, self.LAYER_OUTPUTS[i], bias=False), name=f'weight'))
            layers.append(nn.BatchNorm1d(out_size))
            layers.append(nn.Dropout(f_dropout))
            layers.append(nn.PReLU())
            in_size = out_size

        # Final layer
        layers.append(torch.nn.Linear(in_size, num_targets))
    
        self.model = torch.nn.Sequential(*layers)
        
        # Initialize weights
        self.model.apply(self._init_weights)
        
    def _init_weights(self, m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            if m.bias != None:
                m.bias.data.fill_(0.01)
        
    def forward(self, x):
        x = self.model(x)
        return x

class MoANet2(nn.Module):
    name = "MoANet2"
    def __init__(self, num_features, num_targets):
        super(MoANet2, self).__init__()
        
        hidden_size = 1500
        Dropout_Model = 0.25
        
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(Dropout_Model)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(Dropout_Model)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x
    
class MoANet3(nn.Module):
    name = "MoANet3"
    def __init__(self, num_features, num_targets):
        super(MoANet3, self).__init__()
        
        self.HIDDEN_LAYER_SIZE = 1500
        
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, self.HIDDEN_LAYER_SIZE))
        self.batch_norm1 = nn.BatchNorm1d(self.HIDDEN_LAYER_SIZE)
        self.dropout2 = nn.Dropout(0.25)
        
        self.dense2 = nn.utils.weight_norm(nn.Linear(self.HIDDEN_LAYER_SIZE, self.HIDDEN_LAYER_SIZE))
        self.batch_norm2 = nn.BatchNorm1d(self.HIDDEN_LAYER_SIZE)
        self.dropout3 = nn.Dropout(0.25)
        
        self.dense3 = nn.utils.weight_norm(nn.Linear(self.HIDDEN_LAYER_SIZE, num_targets))
        self.batch_norm3 = nn.BatchNorm1d(num_targets)
    
    def forward(self, x):
        x = self.dense1(x)
        x = self.batch_norm1(x)
        x = F.leaky_relu(x)
        x = self.dropout2(x)
        
        x = self.dense2(x)
        x = self.batch_norm2(x)
        x = F.leaky_relu(x)
        x = self.dropout3(x)
        
        x = self.dense3(x)
        
        return x
    
class MoANet4(nn.Module):
    name = "MoANet4"
    def __init__(self, num_features, num_targets):
        super(MoANet4, self).__init__()
        
        self.HIDDEN_LAYER_SIZE_1 = 1536
        self.HIDDEN_LAYER_SIZE_2 = 2048
        self.HIDDEN_LAYER_SIZE_3 = 1536
        
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, self.HIDDEN_LAYER_SIZE_1))
        
        self.batch_norm2 = nn.BatchNorm1d(self.HIDDEN_LAYER_SIZE_1)
        self.dropout2 = nn.Dropout(0.25)
        self.dense2 = nn.utils.weight_norm(nn.Linear(self.HIDDEN_LAYER_SIZE_1, self.HIDDEN_LAYER_SIZE_2))
        
        self.batch_norm3 = nn.BatchNorm1d(self.HIDDEN_LAYER_SIZE_2)
        self.dropout3 = nn.Dropout(0.25)
        self.dense3 = nn.utils.weight_norm(nn.Linear(self.HIDDEN_LAYER_SIZE_2, self.HIDDEN_LAYER_SIZE_3))
        
        self.batch_norm4 = nn.BatchNorm1d(self.HIDDEN_LAYER_SIZE_3)
        self.dropout4 = nn.Dropout(0.25)
        self.dense4 = nn.utils.weight_norm(nn.Linear(self.HIDDEN_LAYER_SIZE_3, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.leaky_relu(self.dense3(x))
        
        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = self.dense4(x)
        
        return x

class LogisticRegression(nn.Module):
    name = "LogisticRegression"
    def __init__(self, num_features, num_targets):
        super(LogisticRegression, self).__init__()
                
        self.batch_norm = nn.BatchNorm1d(num_features)
        self.dense = nn.utils.weight_norm(nn.Linear(num_features, num_targets))
    
    def forward(self, x):
        x = self.batch_norm(x)
        x = self.dense(x)
        
        return x
    
class WideAndDeepLearning(nn.Module):
    name = "WideAndDeepLearning"
    def __init__(self, num_features, num_targets):
        super(WideAndDeepLearning, self).__init__()
        
        self.HIDDEN_LAYER_SIZE = 2048
        
        hidden_size = self.HIDDEN_LAYER_SIZE
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.wide   = nn.utils.weight_norm(nn.Linear(num_features, num_targets))
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.4)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.4)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
        
        self.batch_norm4 = nn.BatchNorm1d(2 * num_targets)
        self.dropout4 = nn.Dropout(0.4)
        self.dense4 = nn.utils.weight_norm(nn.Linear(2 * num_targets, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x2 = F.leaky_relu(self.dense1(x))
        w = F.leaky_relu(self.wide(x))
        
        x = self.batch_norm2(x2)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        x = w + x
        
        return x
    
class LinearReluBnDropout(nn.Module):
    name = "LinearReluBnDropout"
    def __init__(self, in_features, out_features):
        super(LinearReluBnDropout, self).__init__()
        
        self.DROPOUT = 0.4

        self.block = nn.Sequential(
            nn.utils.weight_norm(nn.Linear(in_features, out_features)),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(out_features),
            nn.Dropout(self.DROPOUT)
        )

    def forward(self, x):
        x = self.block(x)

        return x


class TablarNet(nn.Module):
    name = "TablarNet"
    def __init__(self, num_features, num_targets):
        super(TablarNet, self).__init__()
        
        self.EMBEDDING_DIMENSIONS = [(3, 20), (2, 15)]
        self.DROPOUT = 0.4
        self.HIDDEN_SIZE = 2048
        self.CONTINUOUS_FEATURES = num_features - len(self.EMBEDDING_DIMENSIONS)

        self.embedding_layer = nn.ModuleList([nn.Embedding(x, y) for x, y in self.EMBEDDING_DIMENSIONS])
        self.dropout = nn.Dropout(self.DROPOUT, inplace=True)

        self.first_bn_layer = nn.Sequential(
            nn.BatchNorm1d(self.CONTINUOUS_FEATURES),
            nn.Dropout(self.DROPOUT)
        )

        first_in_feature = self.CONTINUOUS_FEATURES + sum([y for x, y in self.EMBEDDING_DIMENSIONS])

        self.block = nn.Sequential(
            LinearReluBnDropout(in_features=first_in_feature,
                                out_features=self.HIDDEN_SIZE),
            LinearReluBnDropout(in_features=self.HIDDEN_SIZE,
                                out_features=self.HIDDEN_SIZE)
        )

        self.last = nn.Linear(self.HIDDEN_SIZE, num_targets)

    def forward(self, x):
        cont_f = x[:, len(self.EMBEDDING_DIMENSIONS):]
        cat_f = x[:, :len(self.EMBEDDING_DIMENSIONS)]

        cat_x = [layer(cat_f[:, i].long()) for i, layer in enumerate(self.embedding_layer)]
        cat_x = torch.cat(tuple(cat_x), 1)
        cat_x = self.dropout(cat_x)

        cont_x = self.first_bn_layer(cont_f)

        x = torch.cat([cont_x, cat_x], 1)

        x = self.block(x)
        x = self.last(x)

        return x
    
class LightGBM():
    name = "LightGBM"
    def __init__(self):
        self.params = {
            'num_leaves': 24,
            'max_depth': 5,
            'objective': 'binary',
            'learning_rate': 0.01
        }
        self.n_columns = 206
        self.model = [None] * self.n_columns
        
    def fit(self, train_X, train_Y, valid_X, valid_Y):
        for i in range(self.n_columns):
            print(f"#### Training Column {i:>3} ####")
            lgb_train = lgb.Dataset(train_X,
                                         train_Y[:, i])
            lgb_eval  = lgb.Dataset(valid_X,
                                         valid_Y[:, i],
                                         reference=lgb_train)
            self.model[i] = lgb.train(self.params,
                                   lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   verbose_eval=10,
                                   num_boost_round=1000,
                                   early_stopping_rounds=10)
            print(f"#############################")
    def predict(self, X):
        print("Predicting...")
        return np.concatenate(
            [self.model[i].predict(X, 
                                   num_iteration=self.model[i].best_iteration).reshape(-1, 1) for i in range(self.n_columns)],
            axis=1
        )

class XGBoost():
    name = "XGBoost"
    def __init__(self):
        self.n_columns = 206
        self.model = [xgb.XGBClassifier(num_class=2, objective='binary:logistic', verbosity=1)] * self.n_columns
        
    def fit(self, train_X, train_Y, valid_X, valid_Y):
        print(train_X.shape, train_Y.shape)
        print(valid_X.shape, valid_Y.shape)
        for i in range(self.n_columns):
            print(f"#### Training Column {i:>3} ####")
            print(train_X.shape, train_Y[:, i].shape)
            print(valid_X.shape, valid_Y[:, i].shape)
            eval_set = [(valid_X, valid_Y[:, i])]
            self.model[i].fit(train_X, 
                              train_Y[:, i],
                              eval_metric="logloss", 
                              eval_set=eval_set, 
                              early_stopping_rounds=10,
                              verbose=True)
            print(f"#############################")
        
    def predict(self, X):
        print("Predicting...")
        return np.concatenate(
            [self.model[i].predict(X).reshape(-1, 1) for i in range(self.n_columns)],
            axis=1
        )
    
class Tabnet():
    name = "Tabnet"
    def __init__(self):
        self.n_columns = 206
        self.MAX_EPOCH = 200
        # n_d and n_a are different from the original work, 32 instead of 24
        # This is the first change in the code from the original
        self.tabnet_params = dict(
            n_d = 32,
            n_a = 32,
            n_steps = 1,
            gamma = 1.3,
            lambda_sparse = 0,
            optimizer_fn = optim.Adam,
            optimizer_params = dict(lr = 2e-2, weight_decay = 1e-5),
            mask_type = "entmax",
            scheduler_params = dict(
                mode = "min", patience = 5, min_lr = 1e-5, factor = 0.9),
            scheduler_fn = optim.lr_scheduler.ReduceLROnPlateau,
            seed = seed,
            verbose = 10
        )
        self.model = TabNetRegressor(**self.tabnet_params)
        
    def fit(self, train_X, train_Y, valid_X, valid_Y):
        print(train_X.shape, train_Y.shape)
        print(valid_X.shape, valid_Y.shape)
        self.model.fit(
            X_train = train_X,
            y_train = train_Y,
            eval_set = [(valid_X, valid_Y)],
            eval_name = ["val"],
            eval_metric = ["logits_ll"],
            max_epochs = self.MAX_EPOCH,
            patience = 20,
            batch_size = 1024, 
            virtual_batch_size = 32,
            num_workers = 1,
            drop_last = False,
            # To use binary cross entropy because this is not a regression problem
            loss_fn = F.binary_cross_entropy_with_logits
        )
        
    def predict(self, X):
        print("Predicting...")
        preds_val = self.model.predict(X)
        # Apply sigmoid to the predictions
        preds = 1 / (1 + np.exp(-preds_val))
        return preds

In [7]:
def _log_loss(y_true, y_pred):
    score = 0
    for i in range(y_true.shape[1]):
        x = y_true[:, i]
        z = y_pred[:, i]
        score_ = log_loss(x, z, labels=[0, 1])
        score += score_ / y_true.shape[1]

    return score

def _align_features_and_targets(f, t, test_f):
    columns = ["sig_id"]
    columns.extend(TARGET_COLUMNS)
    t = f.merge(t, how="inner", on="sig_id").loc[:, columns]
    if VERBOSE:
        print("Features and Targets Aligned.")

    del columns
    gc.collect()

    return f, t, test_f

def _mapping_and_filter(f, t, test_f):
    cp_type = {'trt_cp': 0, 'ctl_vehicle': 1}
    cp_dose = {'D1': 0, 'D2': 1}
    cp_time = {24: 0, 48: 1, 72:2}
    for df in [f, test_f]:
        df['cp_type'] = df['cp_type'].map(cp_type)
        df['cp_dose'] = df['cp_dose'].map(cp_dose)
        df['cp_time'] = df['cp_time'].map(cp_time)

    if VERBOSE:
        print(f"After Mapping {f.shape[1]} features and {f.shape[0]} training examples present.")
        print("Features Mapped to Integers.")

    del cp_type, cp_dose, cp_time
    gc.collect()

    return f, t, test_f

def _split_with_cp_type(f, t, test_f):
    cp_t = t[f['cp_type'] == 1].reset_index(drop = True)
    cp_f = f[f['cp_type'] == 1].reset_index(drop = True)
    t = t[f['cp_type'] == 0].reset_index(drop = True)
    f = f[f['cp_type'] == 0].reset_index(drop = True)
    cp_f = cp_f.drop("cp_type", axis=1)
    f = f.drop("cp_type", axis=1)
    
    if VERBOSE:
        print(f"After Splitting according to cp_type {f.shape[1]} features and {f.shape[0]} training examples present in first split.")
        print(f"and {cp_f.shape[1]} features and {cp_f.shape[0]} training examples present in second split.")
        print("Examples Split According to cp_type")
    return f, t, test_f, cp_f, cp_t
    

def _scaling(f, t, test_f):
    features = f.columns[3:]
    scaler = RobustScaler()
    scaler.fit(pd.concat([f[features], test_f[features]], axis = 0))
    f[features] = scaler.transform(f[features])
    test_f[features] = scaler.transform(test_f[features])

    if VERBOSE:
        print(f"After Scaling {f.shape[1]} features and {f.shape[0]} training examples present.")
        print("Features Scaled.")

    del features, scaler
    gc.collect()

    return f, t, test_f

def _variance_thresholding(f, t, test_f):
    var_thresh = VarianceThreshold(threshold=VARIANCE_THRESHOLD)

    if VERBOSE:
        print(f"Before Variance Thresholding {f.shape[1]} features present.")


    cols = ['sig_id', 'cp_type', 'cp_time', 'cp_dose']
    data = f.append(test_f)
    var_thresh.fit(data.iloc[:, len(cols):])
    data_transformed = data.iloc[:, len(cols):][data.iloc[:, len(cols):].columns[var_thresh.get_support(indices=True)]]
    train_features_transformed = data_transformed.iloc[:f.shape[0]]
    test_features_transformed = data_transformed.iloc[-test_f.shape[0]:]
    
    train_features = pd.DataFrame(f[cols].values.reshape(-1, len(cols)),
                                  columns=cols)
    f = pd.concat([train_features, train_features_transformed], axis=1)
    test_features = pd.DataFrame(test_f[cols].values.reshape(-1, len(cols)),
                                 columns=cols)
    test_f = pd.concat([test_features, test_features_transformed], axis=1)

    if VERBOSE:
        print(f"After Variance Thresholding {f.shape[1]} features and {f.shape[0]} training examples present.")
        print("Variance Thresholding Done.")

    del data, data_transformed, train_features_transformed, test_features_transformed, train_features, test_features
    gc.collect()

    return f, t, test_f

def _quantile_transform(f, t, test_f):
    QUANTILE_COLUMNS = [c for c in f.columns if c.startswith('c-')] + [c for c in f.columns if c.startswith('g-')] 
#         + [c for c in f.columns if c.startswith('stat-')]
    for col in (QUANTILE_COLUMNS):
        transformer = QuantileTransformer(n_quantiles=50, random_state=SEED, output_distribution="normal")
        vec_len = len(f[col].values)
        vec_len_test = len(test_f[col].values)
        raw_vec = f[col].values.reshape(vec_len, 1)
        transformer.fit(raw_vec)

        f[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
        test_f[col] = transformer.transform(test_f[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

    if VERBOSE:
        print(f"After Quantile Transformation {f.shape[1]} features and {f.shape[0]} training examples present.")
        print("Features Quantile Transformed.")

    del transformer, vec_len, vec_len_test, raw_vec, QUANTILE_COLUMNS
    gc.collect()

    return f, t, test_f

def _add_stats(f, t, test_f):    
    for df in [f, test_f]:
        df['stat-g_sum'] = df[GENES].sum(axis = 1)
        df['stat-g_mean'] = df[GENES].mean(axis = 1)
        df['stat-g_std'] = df[GENES].std(axis = 1)
        df['stat-g_kurt'] = df[GENES].kurtosis(axis = 1)
        df['stat-g_skew'] = df[GENES].skew(axis = 1)
        df['stat-c_sum'] = df[CELLS].sum(axis = 1)
        df['stat-c_mean'] = df[CELLS].mean(axis = 1)
        df['stat-c_std'] = df[CELLS].std(axis = 1)
        df['stat-c_kurt'] = df[CELLS].kurtosis(axis = 1)
        df['stat-c_skew'] = df[CELLS].skew(axis = 1)
        df['stat-gc_sum'] = df[GENES + CELLS].sum(axis = 1)
        df['stat-gc_mean'] = df[GENES + CELLS].mean(axis = 1)
        df['stat-gc_std'] = df[GENES + CELLS].std(axis = 1)
        df['stat-gc_kurt'] = df[GENES + CELLS].kurtosis(axis = 1)
        df['stat-gc_skew'] = df[GENES + CELLS].skew(axis = 1)

    if VERBOSE:
        print(f"After Feature Stats Addition {f.shape[1]} features and {f.shape[0]} training examples present.")
        print("Feature Stats Added.")

    return f, t, test_f


def _bin_columns(f, t, test_f, drop_original=False):
    for col in GENES:
        f.loc[:, f'{col}_bin'] = pd.cut(f[col], bins=3, labels=False)
        test_f.loc[:, f'{col}_bin'] = pd.cut(test_f[col], bins=3, labels=False)

    if drop_original:
        f.drop(GENES).reset_index(drop=True)
        test_f.drop(GENES).reset_index(drop=True)

    if VERBOSE:
        print(f"After Binning {f.shape[1]} features and {f.shape[0]} training examples present.")
        print("Features Binned.")

    return f, t, test_f

def _naive_outlier_removal(f, t, test_f):
    train_ = f.copy() # Didn't wanted to actually normalize, so created a copy and normalized that for further calculation
    for col in GENES:
    #     train_[col] = (train[col]-np.mean(train[col])) / (np.std(train[col]))
        mean = train_[col].mean()
        std = train_[col].std()

        std_r = mean + 4*std
        std_l = mean - 4*std

        drop = train_[col][(train_[col]>std_r) | (train_[col]<std_l)].index.values

    f = f.drop(drop).reset_index(drop=True)
    t = t.drop(drop).reset_index(drop=True)

    if VERBOSE:
        print(f"After Outlier Removal {f.shape[1]} features and {f.shape[0]} training examples present.")
        print("Feature Outliers Removed.")

    del train_, mean, std, std_r, std_l, drop
    gc.collect()

    return f, t, test_f

def _PCA(f, t, test_f, drop_original=False):
    data = pd.concat([pd.DataFrame(f[GENES]), pd.DataFrame(test_f[GENES])])
    data2 = (PCA(n_components=GENE_PCA_COMP, random_state=SEED).fit_transform(data[GENES]))
    train2 = data2[:f.shape[0]]; test2 = data2[-test_f.shape[0]:]

    train2 = pd.DataFrame(train2, columns=[f'tsne_G-{i}' for i in range(GENE_PCA_COMP)])
    test2 = pd.DataFrame(test2, columns=[f'tsne_G-{i}' for i in range(GENE_PCA_COMP)])

    f = pd.concat((f, train2), axis=1)
    test_f = pd.concat((test_f, test2), axis=1)
    
    data = pd.concat([pd.DataFrame(f[CELLS]), pd.DataFrame(test_f[CELLS])])
    data2 = (PCA(n_components=CELL_PCA_COMP, random_state=SEED).fit_transform(data[CELLS]))
    train2 = data2[:f.shape[0]]; test2 = data2[-test_f.shape[0]:]

    train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(CELL_PCA_COMP)])
    test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(CELL_PCA_COMP)])

    f = pd.concat((f, train2), axis=1)
    test_f = pd.concat((test_f, test2), axis=1)

    if drop_original:
        f = f.drop(GENES).reset_index(drop=True)
        f = f.drop(CELLS).reset_index(drop=True)
        test_f = test_f.drop(GENES).reset_index(drop=True)
        test_f = test_f.drop(CELLS).reset_index(drop=True)

    if VERBOSE:
        print(f"After PCA {f.shape[1]} features and {f.shape[0]} training examples present.")
        print("PCA Performed.")            

    return f, t, test_f

def _cluster(f, t, test_f):    
    def create_cluster(train, test, features, kind = 'g', n_clusters = n_clusters_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        kmeans_genes = load(open('../input/moa-tabnet-train-inference/kmeans_genes.pkl', 'rb'))
        train[f'clusters_{kind}'] = kmeans_genes.predict(train_.values)
        test[f'clusters_{kind}'] = kmeans_genes.predict(test_.values)
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
    f, test_f = create_cluster(f, test_f, GENES, kind = 'g', n_clusters = N_CLUSTERS_G)
    f, test_f = create_cluster(f, test_f, GENES, kind = 'c', n_clusters = N_CLUSTERS_C)

    if VERBOSE:
        print(f"After K-Means Clustering {f.shape[1]} features and {f.shape[0]} training examples present.")
        print("K-Means Clustering Performed.")
    return f, t, test_f

In [8]:
def _stratify(f, t, test_f, shuffle=True):
    f.loc[:, "kfold"] = -1
    if shuffle:
        f = f.sample(frac=1).reset_index(drop=True)        

    mskf = MultilabelStratifiedKFold(n_splits=FOLDS)
    for fold_, (train_, val_) in enumerate(mskf.split(X=f, y=t)): 
        f.loc[val_, "kfold"] = fold_

    if VERBOSE:
        print("Dataset Stratified.")
    return f, t, test_f

def _extract_features(f, t, test_f):
    f, t, test_f = _mapping_and_filter(f, t, test_f)
#         f, t, test_f = _bin_columns(f, t, test_f)
#         f, t, test_f = _naive_outlier_removal(f, t, test_f)
#         f, t, test_f = _add_stats(f, t, test_f)
    f, t, test_f = _quantile_transform(f, t, test_f)
    f, t, test_f = _PCA(f, t, test_f, drop_original=False)
    f, t, test_f = _variance_thresholding(f, t, test_f)
#         f, t, test_f = _scaling(f, t, test_f)
    f, t, test_f = _align_features_and_targets(f, t, test_f)
    f, t, test_f, cp_f, cp_t = _split_with_cp_type(f, t, test_f)

    if VERBOSE:
        print("Features Extracted.")
    return f, t, test_f, cp_f, cp_t

In [9]:
PRE_CP_FEATURES_CSV = os.path.join(PATH, "preprocessed_cp_train_features.csv")
PRE_CP_TARGETS_CSV  = os.path.join(PATH, "preprocessed_cp_train_targets.csv")

if USE_SAVED_FOLDS and os.path.isfile(PRE_FEATURES_CSV) and os.path.isfile(PRE_TARGETS_CSV) and os.path.isfile(PRE_TEST_CSV) and os.path.isfile(PRE_CP_FEATURES_CSV) and os.path.isfile(PRE_CP_TARGETS_CSV):
    f = pd.read_csv(PRE_FEATURES_CSV)
    t = pd.read_csv(PRE_TARGETS_CSV)
    test_f = pd.read_csv(PRE_TEST_CSV)
    cp_f = pd.read_csv(PRE_CP_FEATURES_CSV)
    cp_t = pd.read_csv(PRE_CP_TARGETS_CSV)
    
    if VERBOSE:
        print("Saved Folds Loaded.")

else:
    f = pd.read_csv(TRAIN_F)
    t = pd.read_csv(TRAIN_T)
    t_s = pd.read_csv(TRAIN_T_NS)
    test_f = pd.read_csv(TEST_F)

    f, t, test_f, cp_f, cp_t = _extract_features(f, t, test_f)
    f, t, test_f = _stratify(f, t, test_f, shuffle=False)

    if SAVE_FOLDS:
        f.to_csv(PRE_FEATURES_CSV, index=False)
        t.to_csv(PRE_TARGETS_CSV, index=False)
        test_f.to_csv(PRE_TEST_CSV, index=False)
        cp_f.to_csv(PRE_CP_FEATURES_CSV, index=False)
        cp_t.to_csv(PRE_CP_TARGETS_CSV, index=False)

        if VERBOSE:
            print("Folds Saved.")

if VERBOSE:
    print("Dataset Prepared.")

Saved Folds Loaded.
Dataset Prepared.


In [10]:
f

sig_id  cp_time  cp_dose       g-0       g-1       g-2       g-3  \
0      id_000644bb2        0        0  1.132448  0.907530 -0.416136 -0.968385   
1      id_000779bfc        2        0  0.118893  0.680779  0.272846  0.080437   
2      id_000a6266a        1        0  0.779858  0.944713  1.421464 -0.131280   
3      id_0015fd391        1        0 -0.734883 -0.274958 -0.438017  0.758887   
4      id_001626bd3        2        1 -0.451461 -0.476188  0.973532  0.971093   
...             ...      ...      ...       ...       ...       ...       ...   
21943  id_fff8c2444        2        0  0.237214 -1.224115  0.218376 -0.364294   
21944  id_fffb1ceed        0        1  0.208812 -0.022279 -0.236688 -0.796536   
21945  id_fffb70c0c        0        1 -1.893971  0.587155 -0.587397  1.294879   
21946  id_fffcb9e7c        0        0  0.816665  0.416917  0.432679  0.301070   
21947  id_ffffdd77b        2        0 -1.240323  1.565048 -0.269393  1.084626   

            g-4       g-5       g-6       g-7       g-8       g-9      g-10  \
0     -0.254558 -1.016044 -1.362230 -0.025504  0.679664 -0.310373  1.525403   
1      1.205632  0.684773  0.313713  0.554796 -0.537558  0.835997 -1.246502   
2     -0.006836  1.480331  0.234629  0.366079 -0.006818  1.236567 -0.640201   
3      2.125992 -0.858776 -2.060427  0.309097 -0.191607 -1.382882 -0.999761   
4      1.460738 -0.871032 -0.375281 -0.206686 -1.065562  0.849009 -0.335906   
...         ...       ...       ...       ...       ...       ...       ...   
21943 -0.329923  0.569047 -0.153244 -0.246824 -0.936951 -0.030650  0.704618   
21944 -0.674289  0.920115  0.733798  0.507950 -0.624467 -0.042589  0.293488   
21945 -1.003492  0.850209 -0.304510 -0.703869  0.630062  0.644593 -0.292341   
21946  1.070822 -0.023360  0.049912  0.097956  0.530116 -1.190716  0.735723   
21947 -0.511698 -2.059567 -1.616071  1.425946  2.098464  1.253652  2.051259   

           g-11      g-12      g-13      g-14      g-15      g-16      g-17  \
0      0.172311  0.622379 -0.560646  0.280539 -1.057587 -1.138692  0.869104   
1     -0.576923 -0.210653  0.543904  0.147715  0.390836  0.399125  0.397646   
2     -0.770186 -0.130546 -2.080166  0.876118 -0.548526  0.471398 -0.407041   
3     -0.494236 -1.144486 -0.817031 -1.758753  1.393744 -0.228047 -1.051806   
4     -0.703924 -1.247463  1.181020  0.317160  0.342955 -0.025694 -1.560076   
...         ...       ...       ...       ...       ...       ...       ...   
21943 -0.699054  0.446639  0.137659  1.106555 -0.313584  0.108495  0.277079   
21944  0.197849 -0.682393 -1.505804  0.379071 -0.762757  0.227869  0.227497   
21945  1.768590 -0.365457 -0.861488  1.167057  0.968110 -0.496225  0.184463   
21946  0.159009  1.696536  0.245126  0.782638 -2.057513 -0.002779  0.835380   
21947  1.283723  1.851230  1.612093 -0.751622  1.187532 -1.378586  0.032004   

           g-18      g-19      g-20      g-21      g-22      g-23      g-24  \
0      0.380323 -0.509550 -0.725016 -1.332896 -1.664635  1.243729  0.559739   
1     -0.840191  0.497982  1.227924 -0.136634 -0.396518 -0.449514  0.396053   
2     -0.307267 -0.044907 -0.106809 -1.009688 -1.921918  0.533666  0.627196   
3      0.245795 -2.057588  2.051930 -1.354412 -1.100694 -1.060453 -1.419770   
4     -0.849692 -0.706778 -0.446651 -0.291683  1.054738  0.493957  1.681166   
...         ...       ...       ...       ...       ...       ...       ...   
21943  0.052858  0.230946 -0.650065 -0.749883  0.740031 -0.690633  0.191407   
21944 -1.514978 -0.125301  1.125347 -0.507138  0.826290  0.534185 -0.698734   
21945  0.030205  0.192491 -0.290963 -1.119241  0.140501 -1.803182 -0.980897   
21946  0.435184  1.465867 -0.961207  1.376819  0.546953  2.223543  0.630180   
21947 -1.765360 -0.071435 -0.255118 -0.960736 -1.832547 -1.301097 -1.205778   

           g-25      g-26      g-27      g-28      g-29      g-30      g-31  \
0      0.401744  0.240788  0.165177 -0.520293  0.774321  0.414456 -1.102873   
1     -0.584636  0.631916  0.220693

In [11]:
class MetricsCallback(Callback):
    def __init__(self):
        super().__init__()
        self.metrics = []
        self.lr = LEARNING_RATE

    def on_validation_end(self, trainer, pl_module):
        for scheduler in trainer.lr_schedulers:
            param_groups = scheduler['scheduler'].optimizer.param_groups
            lr = param_groups[0]["lr"]
#             if lr < self.lr:
#                 print(f"Learning Rate Reduced to {lr}")
#             if lr > self.lr:
#                 print(f"Learning Rate Increased to {lr}")

            self.lr = lr
        self.metrics.append(trainer.callback_metrics)

class MoALitModule(pl.LightningModule):
    def __init__(self, net, seed, fold):
        super(MoALitModule, self).__init__()
        self.valid_criterion = nn.BCEWithLogitsLoss()
        self.net = net
        self.fold = fold
        self.seed = seed
        self.train_losses = []
        self.valid_losses = []
        self.epoch = 0
        self.best_valid_loss = None
        self.current_epoch_train_loss = None

    def forward(self, x):
        return self.net(x)

#     def configure_optimizers(self):
#         optimizer = torch.optim.Adam(
#             params=self.parameters(), 
#             lr=LEARNING_RATE,
#             weight_decay=WEIGHT_DECAY
#         )
#         scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#             optimizer,
#             patience=0,
#             factor=SCHEDULER_FACTOR,
#             verbose=LEARNING_VERBOSE
#         )
#         return {
#            'optimizer': optimizer,
#            'lr_scheduler': scheduler,
#            'monitor': 'val_loss'
#        }

#     def configure_optimizers(self):
#         optimizer = optim.AdamW(self.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
#         scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=15, eta_min=0)

#         return [optimizer], [scheduler]

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            params=self.parameters(), 
            lr=LEARNING_RATE,
            weight_decay=WEIGHT_DECAY
        )
#         steps_per_epoch = len(self.train_dataloader())
        steps_per_epoch = len(self.train_dataloader())//self.trainer.accumulate_grad_batches
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer=optimizer, 
            pct_start=0.1, 
            div_factor=1e3, 
            max_lr=1e-2,
            steps_per_epoch=steps_per_epoch, 
            epochs=MAX_EPOCHS
        )
        scheduler = {"scheduler": scheduler, "interval" : "step" }
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):        
        inputs, targets = batch
        outputs = self(inputs)

        if WEIGHT is not 1:
            self.train_criterion = SmoothBCEwLogits(smoothing=0.001, weight=((targets * (WEIGHT - 1)) + 1))
        else:
            self.train_criterion = SmoothBCEwLogits(smoothing=0.001)

        loss = self.train_criterion(outputs, targets)

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=False, logger=True)
        self.train_losses.append(loss.item())
        return loss

    def training_epoch_end(self, outputs):
        losses = 0.0
        for output in outputs:
            loss = output["loss"]
            losses += loss.item()

        losses /= len(outputs)

        self.current_epoch_train_loss = losses

    def validation_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = self.valid_criterion(outputs, targets)

        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=False, logger=True)

        self.valid_losses.append(loss.item())

        return loss

    def validation_epoch_end(self, outputs):
        losses = 0.0
        for output in outputs:
            loss = output
            losses += loss.item()

        losses /= len(outputs)

        self.log('val_loss_epoch', loss, on_epoch=True, prog_bar=False, logger=True)

        if self.best_valid_loss is not None:
            self.best_valid_loss = min(self.best_valid_loss, losses)
        else:
            self.best_valid_loss = losses

        if self.current_epoch_train_loss is None:
            print(f"[{self.seed+1:>1}/{SEEDS:>1}][{self.fold+1:>1}/{FOLDS:>1}][Epoch: {self.epoch:>2}] Training Loss: Sanity Check | Validation Loss: {losses:>.10f}", end="")
        else:
            print(f"[{self.seed+1:>1}/{SEEDS:>1}][{self.fold+1:>1}/{FOLDS:>1}][Epoch: {self.epoch:>2}] Training Loss: {self.current_epoch_train_loss:>.10f} | Validation Loss: {losses:>.10f}", end="")
        self.epoch += 1

    def teardown(self, stage):
        train_fold_loss = sum(self.train_losses) / len(self.train_losses)
        valid_fold_loss = sum(self.valid_losses) / len(self.valid_losses)
        best_valid_fold_loss = self.best_valid_loss
        print(f"[{self.seed+1:>1}/{SEEDS:>1}][{self.fold+1:>1}/{FOLDS:>1}] Training Loss: {train_fold_loss:>.10f} | Validation Loss: {valid_fold_loss:>.10f} | Best Validation Loss: {best_valid_fold_loss:>.10f}")

In [12]:
SEEDS             = 1
LEARNING_VERBOSE  = False
BATCH_SIZE        = 128
USE_EARLY_STOPPING= False
EARLY_STOPPING    = 10
LEARNING_RATE     = 1e-3
WEIGHT            = 1
WEIGHT_DECAY      = 1e-5
SCHEDULER_FACTOR  = 0.5
SAVE_TOP_K        = 1
MAX_EPOCHS        = 25

In [13]:
def get_trainer(net, fold):
    tb_logger = pl_loggers.TensorBoardLogger(
        save_dir="tb_logs", 
        name=f"base_fold_{fold}", 
        version=0
    )

    lr_monitor = LearningRateMonitor(
        logging_interval='step'
    )

    metrics_callback = MetricsCallback()

    checkpoint_callback = ModelCheckpoint(
        filepath=f'weights/{net.name}/moa-net={net.name}-fold={fold}-' + '{epoch:03d}-{val_loss_epoch:.4f}',
        monitor='val_loss_epoch',
        save_top_k=SAVE_TOP_K,
        verbose=LEARNING_VERBOSE
    )

    early_stop_callback = EarlyStopping(
        monitor='val_loss',
        min_delta=0.00,
        mode='min',
        patience=EARLY_STOPPING,
        verbose=LEARNING_VERBOSE
    )

    if USE_EARLY_STOPPING:
        trainer = pl.Trainer(
            logger=tb_logger,
            max_epochs=MAX_EPOCHS,
            gpus=1, # -1 if torch.cuda.is_available() else None,
            callbacks=[lr_monitor, metrics_callback, early_stop_callback],
            checkpoint_callback=checkpoint_callback, # Do not save any checkpoints,
        )
    else:
        trainer = pl.Trainer(
            num_sanity_val_steps=0,
            logger=tb_logger,
            max_epochs=MAX_EPOCHS,
            gpus=1, # -1 if torch.cuda.is_available() else None,
            callbacks=[lr_monitor, metrics_callback],
            checkpoint_callback=checkpoint_callback, # Do not save any checkpoints,
        )
    
    return trainer, checkpoint_callback, metrics_callback

In [14]:
def run_fold(net, seed, fold, prediction_ids, submission_ids, prediction, submission, metrics_and_models):
    print(f"Running on Fold #{fold}")
    
    dataset = MoADatasetModule(f=f, t=t, test_f=test_f, fold=fold)
    dataset.prepare_data()
    dataset.setup()

    prediction_ids += dataset.f[dataset.f.kfold==fold]["sig_id"].to_list()
    submission_ids = dataset.test_f["sig_id"].to_list()

    if net not in non_dl_models:
        trainer, checkpoint_callback, metrics_callback = get_trainer(net, fold)

        model = MoALitModule(net(INPUT_SIZE, OUTPUT_SIZE), seed=seed, fold=fold)
        trainer.fit(model, dataset)

        print(f"Best Model for Fold #{fold} saved at {checkpoint_callback.best_model_path}.")
        model = MoALitModule.load_from_checkpoint(checkpoint_callback.best_model_path, net=net(INPUT_SIZE, OUTPUT_SIZE), seed=seed, fold=fold)
        preds = model(torch.tensor(dataset.valid_X.astype("float32"))).sigmoid().detach().cpu().float().numpy()
        subms = model(torch.tensor(dataset.test_X[:, 2:].astype("float32"))).sigmoid().detach().cpu().float().numpy()
        
        best_model_score = checkpoint_callback.best_model_score
#         best_model_score = _log_loss(dataset.valid_Y.astype('float64'),
#                                    model(torch.tensor(dataset.valid_X.astype("float32"))).sigmoid().detach().cpu().float().numpy().astype('float64'))
        metrics_and_models[net.name]["fold_metrics"].append(best_model_score)
        metrics_and_models[net.name]["fold_monitor"].append(metrics_callback)
        metrics_and_models[net.name]["fold_models_path"].append(checkpoint_callback.best_model_path)
        
        del trainer
        gc.collect()

    else:
        model = net()
        model.fit(dataset.train_X.astype("float64"), dataset.train_Y.astype("float64"), dataset.valid_X.astype("float64"), dataset.valid_Y.astype("float64"))
        preds = model.predict(dataset.valid_X.astype("float64"))
        subms = model.predict(dataset.test_X[:, 2:].astype("float64"))
        valid_model_score = _log_loss(dataset.valid_Y.astype('float64'), preds)
        metrics_and_models[net.name]["fold_metrics"].append(valid_model_score)
        
    prediction = np.concatenate([
        prediction,
        preds
    ], axis=0)

    
    submission = (submission + (subms / FOLDS)) if submission is not None else (subms / FOLDS)

    del dataset, model
    gc.collect()
    
    return prediction, submission, prediction_ids, submission_ids, metrics_and_models

In [15]:
def run_seed(seed, net, prediction_ids, submission_ids, prediction, submission, metrics_and_models):
    print(f"Running on Seed Value: {seed}")
    seed_everything(seed)
    
    preds = np.empty((0, len(columns) - 1))
    subms = None
    prediction_ids = []
    submission_ids = []
    
    for fold in folds:
        preds, subms, prediction_ids, submission_ids, metrics_and_models = run_fold(net, seed, fold, 
                                                                                    prediction_ids, 
                                                                                    submission_ids, 
                                                                                    preds, subms, 
                                                                                    metrics_and_models)
        
    
    submission = (submission + (subms / SEEDS)) if submission is not None else (subms / SEEDS)
    prediction = (prediction + (preds / SEEDS)) if prediction is not None else (preds / SEEDS)
    
    return prediction, submission, metrics_and_models, prediction_ids, submission_ids

In [16]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
non_dl_models = [LightGBM, XGBoost, Tabnet]

# nets  = [MoANet1, MoANet2, MoANet3, MoANet4, LogisticRegression, WideAndDeepLearning, TablarNet]
# nets = [LogisticRegression]
nets = [Tabnet]
# nets  = [XGBoost]
seeds = list(range(SEEDS))
folds = list(range(FOLDS))
metrics_and_models = {}
predictions = []
submissions = []

def display_validation_loss():
    avg_validation_loss = 0.0
    for seed in range(SEEDS):
        for fold in range(FOLDS):
            valid_loss = metrics_and_models[net.name]["fold_metrics"][seed * len(folds) + fold]
            print(f"Validation Loss for Seed {seed} Fold {fold}: {valid_loss}")
            avg_validation_loss += valid_loss

    avg_validation_loss /= len(folds) * len(seeds)
    print(f"Average Validation Loss: {avg_validation_loss}")

for net in nets:
    print(f"Training Model : {net.name}")
    PREDICTION = os.path.join(PATH, f"{net.name}-prediction.csv")
    SUBMISSION = os.path.join(PATH, f"{net.name}-submission.csv")
    prediction_ids = []
    submission_ids = []
    columns = ["sig_id"]
    columns.extend(TARGET_COLUMNS)
    
    prediction = None
    submission = None
    
    metrics_and_models[net.name] = {}
    metrics_and_models[net.name]["fold_metrics"] = []
    metrics_and_models[net.name]["fold_models_path"]  = []
    metrics_and_models[net.name]["fold_monitor"] = []
    
    for seed in seeds:
        prediction, submission, metrics_and_models, prediction_ids, submission_ids = run_seed(seed, net, prediction_ids, submission_ids, prediction, submission, metrics_and_models)

    prediction = pd.DataFrame(np.concatenate([np.array(prediction_ids).reshape(-1, 1), prediction], axis=1), columns=columns)
    prediction.to_csv(PREDICTION, index=False)
    submission = pd.DataFrame(np.concatenate([np.array(submission_ids).reshape(-1, 1), submission], axis=1), columns=columns)
    submission.to_csv(SUBMISSION, index=False)
    
    display_validation_loss()
        
    predictions.append(prediction)
    submissions.append(submission)
    del prediction, submission
    gc.collect()

Training Model : Tabnet
Running on Seed Value: 0
Running on Fold #0
Dataset has 1005 features.
Dataset has 206 targets.
Dataset Setup.


KeyboardInterrupt: 

In [17]:
[loss for loss in metrics_and_models['XGBoost']['fold_metrics']]

KeyError: 'XGBoost'

In [18]:
print(nets)
for net in nets:
    avg_validation_loss = 0.0
    for seed in range(SEEDS):
        for fold in range(FOLDS):
            valid_loss = metrics_and_models[net.name]["fold_metrics"][seed * len(folds) + fold]
            avg_validation_loss += valid_loss

    avg_validation_loss /= len(folds) * len(seeds)
    print(f"Average Validation Loss: {avg_validation_loss}")

[<class '__main__.Tabnet'>]


IndexError: list index out of range

In [19]:
feature_df = pd.read_csv(TRAIN_F)
targets_df = pd.read_csv(TRAIN_T)
trt_cp_ids = feature_df[feature_df["cp_type"] == "trt_cp"]["sig_id"].to_list()
ctl_vehicle_ids = feature_df[feature_df["cp_type"] == "ctl_vehicle"]["sig_id"].to_list()
target_cols = targets_df.columns

In [97]:
y_true = targets_df.iloc[:, 1:].to_numpy()

In [98]:
def _add_dropped(p):
    p = targets_df.drop(columns=TARGET_COLUMNS).merge(p, on='sig_id', how='left').fillna(0)
    t = targets_df
    
    return p, t

def _get_weighted(p, w):
    pred = np.zeros((p[0].shape[0], p[0].shape[1] - 1))
    ids = p[0]["sig_id"].to_numpy().reshape(-1, 1)
    for prediction, weight in zip(p, w):
        pred += (prediction.iloc[:, 1:].to_numpy().astype("float64")) * weight
    pred /= sum(w)
    pred = pd.DataFrame(np.concatenate([ids, pred], axis=1), columns=p[0].columns)
    return pred

def _weighted_log_loss(p, w, _t):
    return _log_loss(_t.iloc[:, 1:].to_numpy().astype("float64"), 
                     _get_weighted(p, w).iloc[:, 1:].to_numpy().astype("float64"))

In [99]:
# CPMP's logloss from https://www.kaggle.com/c/lish-moa/discussion/183010
def log_loss_numpy(y_pred):
    y_true_ravel = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = np.where(y_true_ravel == 1, - np.log(y_pred), - np.log(1 - y_pred))
    return loss.mean()

def func_numpy_metric(weights):
    oof_blend = np.tensordot(weights, oof, axes = ((0), (0)))
    return log_loss_numpy(oof_blend)

def grad_func(weights):
    oof_clip = np.clip(oof, 1e-15, 1 - 1e-15)
    gradients = np.zeros(oof.shape[0])
    for i in range(oof.shape[0]):
        a, b, c = y_true, oof_clip[i], np.zeros((oof.shape[1], oof.shape[2]))
        for j in range(oof.shape[0]):
            if j != i:
                c += weights[j] * oof_clip[j]
        gradients[i] = -np.mean((-a*b+(b**2)*weights[i]+b*c)/((b**2)*(weights[i]**2)+2*b*c*weights[i]-b*weights[i]+(c**2)-c))
    return gradients

@njit
def grad_func_jit(weights):
    oof_clip = np.minimum(1 - 1e-15, np.maximum(oof, 1e-15))
    gradients = np.zeros(oof.shape[0])
    for i in range(oof.shape[0]):
        a, b, c = y_true, oof_clip[i], np.zeros((oof.shape[1], oof.shape[2]))
        for j in range(oof.shape[0]):
            if j != i:
                c += weights[j] * oof_clip[j]
        gradients[i] = -np.mean((-a*b+(b**2)*weights[i]+b*c)/((b**2)*(weights[i]**2)+2*b*c*weights[i]-b*weights[i]+(c**2)-c))
    return gradients

In [88]:
nets = [MoANet2, MoANet4, WideAndDeepLearning, LightGBM]
oof  = []
for net in nets:
    PREDICTION = os.path.join(PATH, f"{net.name}-prediction.csv")
    p, mod_t = _add_dropped(pd.read_csv(PREDICTION))
    oof.append(p.iloc[:, 1:].to_numpy())
    loss = log_loss_numpy(p.iloc[:, 1:].to_numpy())
    print(f"Loss for {net.name:<20}: {loss}")

oof.append(np.zeros(oof[0].shape))
oof = np.array(oof)

Loss for MoANet2             : 0.014726273084722978
Loss for MoANet4             : 0.015259071033615297
Loss for WideAndDeepLearning : 0.015237159092215643
Loss for LightGBM            : 0.015978917919369923


In [93]:
test_weights = np.array([1 / oof.shape[0]] * oof.shape[0])

In [94]:
%timeit -r 10 grad_func(test_weights)

3.37 s ± 25.1 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [95]:
%timeit -r 10 grad_func_jit(test_weights)

706 ms ± 9.22 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [100]:
tol = 1e-10
init_guess = [1 / oof.shape[0]] * oof.shape[0]
bnds = [(0, 1) for _ in range(oof.shape[0])]
cons = {'type': 'eq', 
        'fun': lambda x: np.sum(x) - 1, 
        'jac': lambda x: [1] * len(x)}

print('Inital Blend OOF:', func_numpy_metric(init_guess))
start_time = time()
res_scipy = minimize(fun = func_numpy_metric, 
                     x0 = init_guess, 
                     method = 'SLSQP', 
                     jac = grad_func_jit, # grad_func 
                     bounds = bnds, 
                     constraints = cons, 
                     tol = tol)
print(f'[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}] Optimised Blend OOF:', res_scipy.fun)
print('Optimised Weights:', res_scipy.x)

Inital Blend OOF: 0.014906510878240277
[00:31] Optimised Blend OOF: 0.014683531811638466
Optimised Weights: [0.70712037 0.03404589 0.12070993 0.13812382 0.        ]


In [ ]:
for i in res_scipy.x:
    print(i)

In [80]:
w = res_scipy.x.tolist()
w

[0.7071526062599678,
 0.033238741532845166,
 0.12142815607193565,
 0.1381804961352514]

In [ ]:
# Loss for MoANet2 - 7 Seeds 7 Folds : 0.014726273084722966
# Loss for MoANet2                   : 0.015149311005954708
# Loss for LightGBM                  : 0.015978917919369923
# Loss TabNet - 32 Dim               : 0.015279159640954677
# Loss TabNet - 64 Dim               : 0.01527617328348842

In [81]:
p = [pd.read_csv(os.path.join(PATH, f"{net.name}-prediction.csv")) for net in nets]
_t = targets_df
p = [_add_dropped(_p)[0] for _p in p]

In [ ]:
def z(a, b, c):
    return a + b + c

partial_fun = partial(z, 1, 2)
partial_fun(3)

In [ ]:
class OptimizeLogLoss:
    def __init__(self):
        self._w = None
        
    def _get_weighted(self, p, w):
        pred = np.zeros((p[0].shape[0], p[0].shape[1] - 1))
        ids = p[0]["sig_id"].to_numpy().reshape(-1, 1)
        for prediction, weight in zip(p, w):
            pred += (prediction.iloc[:, 1:].to_numpy().astype("float64")) * weight
        pred /= sum(w)
        pred = pd.DataFrame(np.concatenate([ids, pred], axis=1), columns=p[0].columns)
        return pred    
    
    def _weighted_log_loss(self, p, _t, w):
        return _log_loss(_t.iloc[:, 1:].to_numpy().astype("float64"), 
                         self._get_weighted(p, w).iloc[:, 1:].to_numpy().astype("float64"))
    def fit(self, p, _t):
        partial_loss = partial(self._weighted_log_loss, p, _t)
        init_w = np.random.dirichlet(np.ones(len(p)))
        self._w = fmin(partial_loss, init_w, disp=True)
        train_loss = partial_loss(w=self._w)
        print(f"Optimal Weights are {self._w} | Weighted training loss: {partial_loss(w=self._w)}")
        return self._w, train_loss
    
    def validation_loss(self, p, t):
        valid_loss = self._weighted_log_loss(p, t, self._w)
        return valid_loss
        print(f"Optimal Weights are {self._w} | Weighted validation loss: {self._weighted_log_loss(p, t, self._w)}")
    
    def predict(self, p):
        return self._get_weighted(p, self._w)

In [ ]:
optimizer = OptimizeLogLoss()
mskf = MultilabelStratifiedKFold(n_splits=FOLDS)
w = []
for fold_, (train_, valid_) in enumerate(mskf.split(X=_t, y=_t)): 
    p_t = [_p.iloc[train_, :] for _p in p]
    p_v = [_p.iloc[valid_, :] for _p in p]
    t_t = _t.iloc[train_, :]
    t_v = _t.iloc[valid_, :]
    _w, train_loss = optimizer.fit(p_t, t_t)
    valid_loss = optimizer.validation_loss(p_v, t_v)
    print(f"Optimal Weights are {_w} | Weighted Training loss: {_weighted_log_loss(p, t, self._w)} | Weighted Validation loss: {_weighted_log_loss(p, t, self._w)}")
    w.append(_w)

In [ ]:
w = np.mean(np.array(w), axis = 0)
w

In [82]:
_t

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [83]:
p[0]

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [84]:
p[1]

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [85]:
_weighted_log_loss(p, w, _t)

0.014683533165345411

In [86]:
s = [pd.read_csv(os.path.join(PATH, f"{net.name}-submission.csv")) for net in nets]
submission_final = _get_weighted(s, w)
submission_final

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [87]:
submission_final.to_csv(os.path.join(PATH, f"submission.csv"), index=False)